### Importing the Libraries

In [20]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [21]:
tf.__version__ 

'2.15.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [22]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [23]:
test_datagen = ImageDataGenerator(rescale = 1./255)

### Creating the Training set

In [24]:
training_set = train_datagen.flow_from_directory('D:\\STTC\\dataSet\\trainingData',                                
                                                 target_size = (85, 85),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 12845 images belonging to 27 classes.


In [25]:
test_set = test_datagen.flow_from_directory('D:\\STTC\\dataSet\\testingData',
                                            target_size = (85, 85),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 1159 images belonging to 27 classes.


### Part 2 - Building the CNN

#### Initializing the CNN

In [26]:
classifier = tf.keras.models.Sequential()

#### Step 1 - Convolution

In [27]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[85, 85, 1]))

#### Step 2 - Pooling

In [28]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Adding a second convolutional layer

In [29]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Step 3 - Flattening

In [30]:
classifier.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [31]:
classifier.add(tf.keras.layers.Dense(units=85, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=27, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [32]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [33]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 85, 85, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 42, 42, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 42, 42, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 21, 21, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 14112)             0         
                                                                 
 dense_4 (Dense)             (None, 85)               

In [34]:
classifier.fit(training_set,
                  epochs = 6,
                  validation_data = test_set)

Epoch 1/6
1193/1285 [==========================>...] - ETA: 2s - loss: 2.8898 - accuracy: 0.1359

#### Saving the Model

In [23]:
model_json = classifier.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_new.h5')
print('Weights saved')

Model Saved
Weights saved
